In [2]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [4]:
img_height = 28 
img_width = 28 
batch_size = 32

# Loading the dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "./mnist_dataset/",
    labels="inferred",
    label_mode="int",
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training"
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "./mnist_dataset/",
    labels="inferred",
    label_mode="int",
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation"
)

# Apply image normalization
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

Found 350 files belonging to 10 classes.
Using 280 files for training.
Found 350 files belonging to 10 classes.
Using 70 files for validation.



In [ ]:
train_processed = []
for images, labels in train_ds:
    for image in images:
        # Convert the image to numpy array
        image_np = image.numpy()

        # Apply contour detection
        contours, _ = cv2.findContours(image_np, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Sort contours from left to right
        contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])

        # Iterate over the contours
        for contour in contours:
            # Get the bounding rectangle for the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Crop the digit from the image
            digit = image_np[y:y+h, x:x+w]

            # Resize the digit to 28x28
            digit_resized = cv2.resize(digit, (28, 28))

            # Normalize the pixel values
            digit_normalized = digit_resized / 255.0

            # Append the processed digit to the list
            train_processed.append((digit_normalized, labels))

# Convert the processed data to numpy arrays
train_processed = np.array(train_processed)

In [ ]:
test_processed = []
for images, labels in test_ds:
    for image in images:
        # Convert the image to numpy array
        image_np = image.numpy()

        # Apply contour detection
        _, contours, _ = cv2.findContours(image_np, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Sort contours from left to right
        contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])

        # Iterate over the contours
        for contour in contours:
            # Get the bounding rectangle for the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Crop the digit from the image
            digit = image_np[y:y+h, x:x+w]

            # Resize the digit to 28x28
            digit_resized = cv2.resize(digit, (28, 28))

            # Normalize the pixel values
            digit_normalized = digit_resized / 255.0

            # Append the processed digit to the list
            test_processed.append((digit_normalized, labels))

# Convert the processed data to numpy arrays
test_processed = np.array(test_processed)

In [ ]:

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))


def predict_number(image):
    image = image.reshape(1, 28, 28, 1) / 255.0
    prediction = model.predict(image)
    digit = tf.argmax(prediction, axis=1).numpy()[0]
    return digit

num = 5
def predict_multidigit_number(image):
    digits = []
    for i in range(num):
        digit = predict_number(image)
        digits.append(digit)
        image = image[:, :-10, :]
    digits.reverse()
    number = sum([digit * (10 ** i) for i, digit in enumerate(digits)])
    return number


test_image = x_test[0]
predicted_number = predict_multidigit_number(test_image)
print(f'Predicted Number: {predicted_number}')